In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [2]:
df = spark.read.format("csv").load("organizations.csv", header=True, inferSchema=True)

df.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Organization Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Website: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Founded: integer (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Number of employees: integer (nullable = true)



In [3]:
df.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|               China|Switchable scalab...|   1971|       Public Safety|               5287|
|    4|2bFC1Be8a4ce42f|      Holder-Sellers| https://becke

In [4]:
df.columns

['Index',
 'Organization Id',
 'Name',
 'Website',
 'Country',
 'Description',
 'Founded',
 'Industry',
 'Number of employees']

In [5]:
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn('id', monotonically_increasing_id())

df = df[['id'] + df.columns[:-1]]

df.show(6)

+---+-----+---------------+--------------------+--------------------+----------------+--------------------+-------+--------------------+-------------------+
| id|Index|Organization Id|                Name|             Website|         Country|         Description|Founded|            Industry|Number of employees|
+---+-----+---------------+--------------------+--------------------+----------------+--------------------+-------+--------------------+-------------------+
|  0|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|
|  1|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|         Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|
|  2|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|           China|Switchable scalab...|   1971|       Public Safety|               5287|
|  3|    4|2bFC1Be8a4ce42f|      Holder-Sellers| https://b

In [6]:
df.count()

100

In [7]:
df.select('Number of employees').agg({'Number of employees': 'avg'}).show()

+------------------------+
|avg(Number of employees)|
+------------------------+
|                 4964.86|
+------------------------+



In [8]:
from pyspark.sql.functions import mean

df.select(*[mean(c) for c in df.columns]).show()

+-------+----------+--------------------+---------+------------+------------+----------------+------------+-------------+------------------------+
|avg(id)|avg(Index)|avg(Organization Id)|avg(Name)|avg(Website)|avg(Country)|avg(Description)|avg(Founded)|avg(Industry)|avg(Number of employees)|
+-------+----------+--------------------+---------+------------+------------+----------------+------------+-------------+------------------------+
|   49.5|      50.5|                null|     null|        null|        null|            null|     1995.41|         null|                 4964.86|
+-------+----------+--------------------+---------+------------+------------+----------------+------------+-------------+------------------------+



In [9]:
df.groupby('Country').agg({col: 'avg' for col in df.columns[3:-1]}).show()

+--------------------+---------+------------+------------------+----------------+-------------+------------+
|             Country|avg(Name)|avg(Country)|      avg(Founded)|avg(Description)|avg(Industry)|avg(Website)|
+--------------------+---------+------------+------------------+----------------+-------------+------------+
|                Chad|     null|        null|            1997.0|            null|         null|        null|
|            Anguilla|     null|        null|            1971.0|            null|         null|        null|
|Heard Island and ...|     null|        null|            1970.0|            null|         null|        null|
|              Sweden|     null|        null|1996.3333333333333|            null|         null|        null|
|French Southern T...|     null|        null|            2021.0|            null|         null|        null|
|         Philippines|     null|        null|            2021.0|            null|         null|        null|
|              Jers

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Index", "Founded"], outputCol="Independent Features")

In [11]:
output=featureassembler.transform(df)

In [12]:
output.show()

+---+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------------------+
| id|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Independent Features|
+---+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------------------+
|  0|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|        [1.0,1990.0]|
|  1|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|        [2.0,2015.0]|
|  2|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|               China|

In [13]:
output.columns

['id',
 'Index',
 'Organization Id',
 'Name',
 'Website',
 'Country',
 'Description',
 'Founded',
 'Industry',
 'Number of employees',
 'Independent Features']

In [14]:
finalized_data=output.select("Independent Features", "Founded")

In [15]:
finalized_data.show()

+--------------------+-------+
|Independent Features|Founded|
+--------------------+-------+
|        [1.0,1990.0]|   1990|
|        [2.0,2015.0]|   2015|
|        [3.0,1971.0]|   1971|
|        [4.0,2004.0]|   2004|
|        [5.0,1991.0]|   1991|
|        [6.0,1992.0]|   1992|
|        [7.0,2018.0]|   2018|
|        [8.0,1970.0]|   1970|
|        [9.0,1996.0]|   1996|
|       [10.0,1997.0]|   1997|
|       [11.0,2001.0]|   2001|
|       [12.0,2014.0]|   2014|
|       [13.0,2020.0]|   2020|
|       [14.0,2013.0]|   2013|
|       [15.0,1984.0]|   1984|
|       [16.0,1990.0]|   1990|
|       [17.0,1972.0]|   1972|
|       [18.0,1998.0]|   1998|
|       [19.0,1999.0]|   1999|
|       [20.0,2011.0]|   2011|
+--------------------+-------+
only showing top 20 rows



In [22]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data, test_data=finalized_data.randomSplit([0.70,0.30])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Founded')
regressor=regressor.fit(train_data)

In [23]:
### Coefficients
regressor.coefficients

DenseVector([-0.0, 1.0])

In [24]:
### Intercepts
regressor.intercept

1.9620838550216413e-09

In [25]:
### Predicition
pred_results=regressor.evaluate(test_data)

In [26]:
pred_results.predictions.show(18)

+--------------------+-------+------------------+
|Independent Features|Founded|        prediction|
+--------------------+-------+------------------+
|        [8.0,1970.0]|   1970|1970.0000000000296|
|       [10.0,1997.0]|   1997|1997.0000000000027|
|       [13.0,2020.0]|   2020|  2019.99999999998|
|       [16.0,1990.0]|   1990|1990.0000000000089|
|       [21.0,1986.0]|   1986|1986.0000000000123|
|       [23.0,2014.0]|   2014|2013.9999999999845|
|       [25.0,2002.0]|   2002|2001.9999999999961|
|       [33.0,1972.0]|   1972|1972.0000000000246|
|       [34.0,1981.0]|   1981|1981.0000000000155|
|       [35.0,2020.0]|   2020| 2019.999999999977|
|       [36.0,2010.0]|   2010|2009.9999999999868|
|       [37.0,2013.0]|   2013|2012.9999999999839|
|       [39.0,2012.0]|   2012|2011.9999999999845|
|       [41.0,2006.0]|   2006|2005.9999999999902|
|       [43.0,2021.0]|   2021|2020.9999999999752|
|       [45.0,1986.0]|   1986|1986.0000000000093|
|       [47.0,1976.0]|   1976|1976.0000000000189|


In [21]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1.3339255626002947e-11, 2.2410467336390365e-22)